In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta

yf.pdr_override()

In [2]:
#测试
start="2023-01-01"
end="2023-09-21"
yoAPL = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)

In [3]:
def pct_change(data):
    data['Close_price_percentage'] = data['Close'].pct_change()
    data['real_situation'] = 0
    data.loc[data['Close_price_percentage'] > 0,'real_situation'] = "涨"
    data.loc[data['Close_price_percentage'] < 0,'real_situation'] ='跌'
    data.loc[data['Close_price_percentage'] == 0,'real_situation'] ='不变'
    return data




In [4]:
def MACD(data):
    data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['DIF'] = data['EMA12'] - data['EMA26']
    data['MACD'] = data['DIF'].ewm(span=9, adjust=False).mean()
    data['Position'] = 0
    data.loc[data['DIF'] > data['MACD'],'Position'] = 1
    data.loc[data['DIF'] < data['MACD'],'Position'] = -1
    return data

def MACD_short(data):
    data['EMA6'] = data['Close'].ewm(span=6, adjust=False).mean()
    data['EMA19'] = data['Close'].ewm(span=19, adjust=False).mean()
    data['DIF'] = data['EMA6'] - data['EMA19']
    data['MACD'] = data['DIF'].ewm(span=9, adjust=False).mean()
    data['Position'] = 0
    data.loc[data['DIF'] > data['MACD'],'Position'] = 1
    data.loc[data['DIF'] < data['MACD'],'Position'] = -1
    return data

def MACD_long(data):
    data['EMA19'] = data['Close'].ewm(span=19, adjust=False).mean()
    data['EMA39'] = data['Close'].ewm(span=39, adjust=False).mean()
    data['DIF'] = data['EMA19'] - data['EMA39']
    data['MACD'] = data['DIF'].ewm(span=9, adjust=False).mean()
    data['Position'] = 0
    data.loc[data['DIF'] > data['MACD'],'Position'] = 1
    data.loc[data['DIF'] < data['MACD'],'Position'] = -1
    return data


def RSI(data):
    # 短期
    rsi_6 = talib.RSI(data["Close"],timeperiod=6)
    data["RSI6"]=rsi_6
    # 中期
    rsi_14 = talib.RSI(data["Close"],timeperiod=14)
    data["RSI14"]=rsi_14
    #长期
    rsi_21 = talib.RSI(data["Close"],timeperiod=21)
    data["RSI21"]=rsi_21
    return data

def RSI6_MACD8020(data):
    data.loc[data['RSI6'] > 80,'Position'] = -1
    data.loc[data['RSI6'] < 20,'Position'] = 1
    return data

def RSI6_MACD7030(data):
    data.loc[data['RSI6'] > 70,'Position'] = -1
    data.loc[data['RSI6'] < 30,'Position'] = 1
    return data

def RSI14_MACD8020(data):
    data.loc[data['RSI14'] > 80,'Position'] = -1
    data.loc[data['RSI14'] < 20,'Position'] = 1
    return data

def RSI14_MACD7030(data):
    data.loc[data['RSI14'] > 70,'Position'] = -1
    data.loc[data['RSI14'] < 30,'Position'] = 1
    return data

def RSI21_MACD8020(data):
    data.loc[data['RSI21'] > 80,'Position'] = -1
    data.loc[data['RSI21'] < 20,'Position'] = 1
    return data

def RSI21_MACD7030(data):
    data.loc[data['RSI21'] > 70,'Position'] = -1
    data.loc[data['RSI21'] < 30,'Position'] = 1
    return data

In [5]:

def results(data):
    close_price=list(data["Close"])
    signal=list(data["Position"])
    real_situation=list(data["real_situation"])
    total_day=len(real_situation)
    # 一直持有收益
    #假设持有1000股
    first_day =close_price[0]
    last_day =close_price[-1]
    value_diff=(last_day-first_day)*1000
    
    #prediction_rate
    i=0
    count=0
    while i<len(real_situation):
        if real_situation[i]=="涨" and signal[i]==1:
            count+=1
        elif real_situation[i]=="跌" and signal[i]==-1:
            count+=1
        i+=1
    rate=count/total_day
   
    #策略收益
    i=1
    revenue=0
    buy_price=close_price[0]
    
    while i<len(close_price)-1:
        if signal[i]==0:
            i+=1
        
        elif signal[i]==1 and signal[i+1]==1:
            
            i+=1
        elif signal[i]==1 and signal[i+1]==-1:
            profit=close_price[i+1]-buy_price
            revenue+= profit
            
            i+=1
        elif signal[i]==-1 and signal[i+1]==-1:
            
            i+=1
        elif signal[i]==-1 and signal[i+1]==1:
            buy_price=close_price[i+1]
            
            i+=1
    strategy=revenue*1000
   
    return rate,value_diff,strategy



In [6]:
def days(months):
    day_collector=[]
    i=1
    while i<months:
        #current_date = datetime.now()
        current_date = datetime(year=2023, month=9, day=20)
        previous_date = current_date - relativedelta(months=i)
        previous_date  = previous_date.strftime("%Y-%m-%d")
        day_collector.append(previous_date)
        i+=1
    return day_collector
    
day=days(30)
print(day)

#print(formatted_date,previous_date)
#for date in start_date:

['2023-08-20', '2023-07-20', '2023-06-20', '2023-05-20', '2023-04-20', '2023-03-20', '2023-02-20', '2023-01-20', '2022-12-20', '2022-11-20', '2022-10-20', '2022-09-20', '2022-08-20', '2022-07-20', '2022-06-20', '2022-05-20', '2022-04-20', '2022-03-20', '2022-02-20', '2022-01-20', '2021-12-20', '2021-11-20', '2021-10-20', '2021-09-20', '2021-08-20', '2021-07-20', '2021-06-20', '2021-05-20', '2021-04-20']


In [ ]:

rate1=[]
value_diff1=[]
strategy1=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    result=results(stock_data)
    rate1.append(result[0])
    value_diff1.append(result[1])
    strategy1.append(result[2])
    
#MACD_6_19
rate2=[]
value_diff2=[]
strategy2=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate2.append(result[0])
    value_diff2.append(result[1])
    strategy2.append(result[2])
    
#MACD_19_39
rate3=[]
value_diff3=[]
strategy3=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate3.append(result[0])
    value_diff3.append(result[1])
    strategy3.append(result[2])
    
#MACD619RSI6_8020
rate4=[]
value_diff4=[]
strategy4=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI6_MACD8020(stock_data)
    
    result=results(stock_data)
    rate4.append(result[0])
    value_diff4.append(result[1])
    strategy4.append(result[2])

#MACD619RSI6_7030
rate5=[]
value_diff5=[]
strategy5=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI6_MACD7030(stock_data)
    
    result=results(stock_data)
    rate5.append(result[0])
    value_diff5.append(result[1])
    strategy5.append(result[2])
    
    
#MACD619RSI14_8020
rate6=[]
value_diff6=[]
strategy6=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI14_MACD8020(stock_data)
    
    result=results(stock_data)
    rate6.append(result[0])
    value_diff6.append(result[1])
    strategy6.append(result[2])
    
#MACD619RSI14_7030
rate7=[]
value_diff7=[]
strategy7=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI14_MACD7030(stock_data)
    
    result=results(stock_data)
    rate7.append(result[0])
    value_diff7.append(result[1])
    strategy7.append(result[2])
    
#MACD619RSI21_8020
rate8=[]
value_diff8=[]
strategy8=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI21_MACD8020(stock_data)
    
    result=results(stock_data)
    rate8.append(result[0])
    value_diff8.append(result[1])
    strategy8.append(result[2])
    
#MACD619RSI21_7030
rate9=[]
value_diff9=[]
strategy9=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI21_MACD7030(stock_data)
    
    result=results(stock_data)
    rate9.append(result[0])
    value_diff9.append(result[1])
    strategy9.append(result[2])

'''
#######################
#MACD1939RSI6_8020
rate10=[]
value_diff10=[]
strategy10=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI6_MACD8020(stock_data)
    
    result=results(stock_data)
    rate10.append(result[0])
    value_diff10.append(result[1])
    strategy10.append(result[2])

#MACD619RSI6_7030
rate11=[]
value_diff11=[]
strategy11=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI6_MACD7030(stock_data)
    
    result=results(stock_data)
    rate11.append(result[0])
    value_diff11.append(result[1])
    strategy11.append(result[2])
    
    
#MACD619RSI14_8020
rate12=[]
value_diff12=[]
strategy12=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI14_MACD8020(stock_data)
    
    result=results(stock_data)
    rate12.append(result[0])
    value_diff12.append(result[1])
    strategy12.append(result[2])
    
#MACD619RSI14_7030
rate13=[]
value_diff13=[]
strategy13=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI14_MACD7030(stock_data)
    
    result=results(stock_data)
    rate13.append(result[0])
    value_diff13.append(result[1])
    strategy13.append(result[2])
  
#MACD619RSI21_8020
rate14=[]
value_diff14=[]
strategy14=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI21_MACD8020(stock_data)
    
    result=results(stock_data)
    rate14.append(result[0])
    value_diff14.append(result[1])
    strategy14.append(result[2])
    
#MACD619RSI21_7030
rate15=[]
value_diff15=[]
strategy15=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI21_MACD7030(stock_data)
    
    result=results(stock_data)
    rate9.append(result[0])
    value_diff9.append(result[1])
    strategy9.append(result[2])
########################################
#MACD1226RSI6_8020
rate16=[]
value_diff16=[]
strategy16=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI6_MACD8020(stock_data)
    
    result=results(stock_data)
    rate16.append(result[0])
    value_diff16.append(result[1])
    strategy16.append(result[2])

#MACD1226RSI6_7030
rate17=[]
value_diff17=[]
strategy17=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI6_MACD7030(stock_data)
    
    result=results(stock_data)
    rate17.append(result[0])
    value_diff17.append(result[1])
    strategy17.append(result[2])
    
    
#MACD1226RSI14_8020
rate18=[]
value_diff18=[]
strategy18=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI14_MACD8020(stock_data)
    
    result=results(stock_data)
    rate18.append(result[0])
    value_diff18.append(result[1])
    strategy18.append(result[2])
    
#MACD1226RSI14_7030
rate19=[]
value_diff19=[]
strategy19=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI14_MACD7030(stock_data)
    
    result=results(stock_data)
    rate19.append(result[0])
    value_diff19.append(result[1])
    strategy19.append(result[2])
    
#MACD1226RSI21_8020
rate20=[]
value_diff20=[]
strategy20=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI21_MACD8020(stock_data)
    
    result=results(stock_data)
    rate20.append(result[0])
    value_diff20.append(result[1])
    strategy20.append(result[2])
    
#MACD1226RSI21_7030
rate21=[]
value_diff21=[]
strategy21=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AAPL',start=start,end=end,progress=False)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI21_MACD7030(stock_data)
    
    result=results(stock_data)
    rate21.append(result[0])
    value_diff21.append(result[1])
    strategy21.append(result[2])
'''

In [ ]:
plt.figure(figsize=(40,20))

plt.plot(value_diff1,label="hold")
plt.plot(strategy1,label="12_26")
plt.plot(strategy2,label="6_19")
plt.plot(strategy3,label="19_39")
plt.plot(strategy4,label="rsi68020")
plt.plot(strategy5,label="rsi67030")
plt.plot(strategy6,label="6")
plt.plot(strategy7,label="7")
plt.plot(strategy8,label="8")
plt.plot(strategy9,label="9")

'''

plt.plot(strategy10,label="10")
plt.plot(strategy11,label="11")
plt.plot(strategy12,label="12")
plt.plot(strategy13,label="13")
plt.plot(strategy14,label="14")
plt.plot(strategy15,label="15")
plt.plot(strategy16,label="16")
plt.plot(strategy17,label="17")
plt.plot(strategy18,label="18")
plt.plot(strategy19,label="19")
plt.plot(strategy20,label="20")
plt.plot(strategy21,label="21")
'''
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate1,label="12_26")
plt.plot(rate2,label="6_19")
plt.plot(rate3,label="19_39")
plt.plot(rate4,label="rsi68020")
plt.plot(rate5,label="rsi67030")
plt.plot(rate6,label="6")
plt.plot(rate7,label="7")
plt.plot(rate8,label="8")
plt.plot(rate9,label="9")
'''
plt.plot(rate10,label="10")
plt.plot(rate11,label="11")
plt.plot(rate12,label="12")
plt.plot(rate13,label="13")
plt.plot(rate14,label="14")
plt.plot(rate15,label="15")
plt.plot(rate16,label="16")
plt.plot(rate17,label="17")
plt.plot(rate18,label="18")
plt.plot(rate19,label="19")
plt.plot(rate20,label="20")
plt.plot(rate21,label="21")

'''
plt.legend()
plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
www

In [ ]:
rate=[]
value_diff=[]
strategy=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='GOOG',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    
    result=results(stock_data)
    rate.append(result[0])
    value_diff.append(result[1])
    strategy.append(result[2])
   
rate_s=[]
value_diff_s=[]
strategy_s=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='GOOG',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate_s.append(result[0])
    value_diff_s.append(result[1])
    strategy_s.append(result[2])
   
    
rate_l=[]
value_diff_l=[]
strategy_l=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='GOOG',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate_l.append(result[0])
    value_diff_l.append(result[1])
    strategy_l.append(result[2])
    


In [ ]:
plt.figure(figsize=(20,10))

plt.plot(value_diff,label="hold")
plt.plot(strategy,label="12_26")
plt.plot(strategy_s,label="6_19")
plt.plot(strategy_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate,label="12_26")
plt.plot(rate_s,label="6_19")
plt.plot(rate_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
rate=[]
value_diff=[]
strategy=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AMZN',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    
    result=results(stock_data)
    rate.append(result[0])
    value_diff.append(result[1])
    strategy.append(result[2])
    

rate_s=[]
value_diff_s=[]
strategy_s=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AMZN',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate_s.append(result[0])
    value_diff_s.append(result[1])
    strategy_s.append(result[2])
    
    
rate_l=[]
value_diff_l=[]
strategy_l=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='AMZN',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate_l.append(result[0])
    value_diff_l.append(result[1])
    strategy_l.append(result[2])
    


In [ ]:
plt.figure(figsize=(20,10))

plt.plot(value_diff,label="hold")
plt.plot(strategy,label="12_26")
plt.plot(strategy_s,label="6_19")
plt.plot(strategy_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate,label="12_26")
plt.plot(rate_s,label="6_19")
plt.plot(rate_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
rate=[]
value_diff=[]
strategy=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='TSLA',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    
    result=results(stock_data)
    rate.append(result[0])
    value_diff.append(result[1])
    strategy.append(result[2])
    

rate_s=[]
value_diff_s=[]
strategy_s=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='TSLA',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate_s.append(result[0])
    value_diff_s.append(result[1])
    strategy_s.append(result[2])
    
    
rate_l=[]
value_diff_l=[]
strategy_l=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='TSLA',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate_l.append(result[0])
    value_diff_l.append(result[1])
    strategy_l.append(result[2])
    


In [ ]:
plt.figure(figsize=(20,10))

plt.plot(value_diff,label="hold")
plt.plot(strategy,label="12_26")
plt.plot(strategy_s,label="6_19")
plt.plot(strategy_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate,label="12_26")
plt.plot(rate_s,label="6_19")
plt.plot(rate_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
rate=[]
value_diff=[]
strategy=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='BABA',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    
    result=results(stock_data)
    rate.append(result[0])
    value_diff.append(result[1])
    strategy.append(result[2])
    

rate_s=[]
value_diff_s=[]
strategy_s=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='BABA',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate_s.append(result[0])
    value_diff_s.append(result[1])
    strategy_s.append(result[2])
    
    
rate_l=[]
value_diff_l=[]
strategy_l=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='BABA',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate_l.append(result[0])
    value_diff_l.append(result[1])
    strategy_l.append(result[2])
    


In [ ]:
plt.figure(figsize=(20,10))

plt.plot(value_diff,label="hold")
plt.plot(strategy,label="12_26")
plt.plot(strategy_s,label="6_19")
plt.plot(strategy_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate,label="12_26")
plt.plot(rate_s,label="6_19")
plt.plot(rate_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
rate=[]
value_diff=[]
strategy=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='0700.HK',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    
    result=results(stock_data)
    rate.append(result[0])
    value_diff.append(result[1])
    strategy.append(result[2])
    

rate_s=[]
value_diff_s=[]
strategy_s=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='0700.HK',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate_s.append(result[0])
    value_diff_s.append(result[1])
    strategy_s.append(result[2])
    
    
rate_l=[]
value_diff_l=[]
strategy_l=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='0700.HK',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate_l.append(result[0])
    value_diff_l.append(result[1])
    strategy_l.append(result[2])
   


In [ ]:
plt.figure(figsize=(20,10))

plt.plot(value_diff,label="hold")
plt.plot(strategy,label="12_26")
plt.plot(strategy_s,label="6_19")
plt.plot(strategy_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate,label="12_26")
plt.plot(rate_s,label="6_19")
plt.plot(rate_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
rate=[]
value_diff=[]
strategy=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='1211.HK',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD(stock_data)
    
    result=results(stock_data)
    rate.append(result[0])
    value_diff.append(result[1])
    strategy.append(result[2])
    

rate_s=[]
value_diff_s=[]
strategy_s=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='1211.HK',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_short(stock_data)
    
    result=results(stock_data)
    rate_s.append(result[0])
    value_diff_s.append(result[1])
    strategy_s.append(result[2])
    
    
rate_l=[]
value_diff_l=[]
strategy_l=[]
for date in day:
    start=date
    end="2023-09-21"
    stock_data = data.get_data_yahoo(tickers='1211.HK',start=start,end=end)
    stock_data=pct_change(stock_data)
    stock_data=MACD_long(stock_data)
    
    result=results(stock_data)
    rate_l.append(result[0])
    value_diff_l.append(result[1])
    strategy_l.append(result[2])
    


In [ ]:
plt.figure(figsize=(20,10))

plt.plot(value_diff,label="hold")
plt.plot(strategy,label="12_26")
plt.plot(strategy_s,label="6_19")
plt.plot(strategy_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(rate,label="12_26")
plt.plot(rate_s,label="6_19")
plt.plot(rate_l,label="19_39")
plt.legend()

plt.title("Line Chart")
plt.xlabel("Date")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
预报涨跌幅度
加指标
要多用股票，比较
每只股票结果不一样
capm
股票入场 资金
短期长期 
min/standard_division /跟好252
sharpe ratio
选波动
maximum drawdown

